In [56]:
pip install webdriver-manager

  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.


In [152]:
import warnings
warnings.filterwarnings(action='ignore')
import time
import sys
import os
import yfinance as yf
import FinanceDataReader as fdr
import pymysql
from bs4 import BeautifulSoup
import requests
from requests_html import HTMLSession
from datetime import datetime, timedelta
from dotenv import load_dotenv
import google.generativeai as genai
import numpy as np
import pandas as pd

In [96]:
news = yf.Ticker("NVDA").news
news

[{'uuid': '8219be13-574c-3850-8220-48cb2256fc8e',
  'title': 'Is It Too Late to Buy Broadcom Stock After Its 10-for-1 Split?',
  'publisher': 'Motley Fool',
  'link': 'https://finance.yahoo.com/m/8219be13-574c-3850-8220-48cb2256fc8e/is-it-too-late-to-buy.html',
  'providerPublishTime': 1721745900,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/A11j3wGAcdKDfp8oCbNCnA--~B/aD00NDE7dz03MjA7YXBwaWQ9eXRhY2h5b24-/https://media.zenfs.com/en/motleyfool.com/f10db056c56564c3de7127f2749ea481',
     'width': 720,
     'height': 441,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/FlKUdlgDlt3Gc7Vwbf.pJg--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/motleyfool.com/f10db056c56564c3de7127f2749ea481',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['AVGO', '^GSPC', 'TSLA', 'NVDA']},
 {'uuid': 'd29d21a2-43ab-3ca8-8e5e-f8e61caa5b70',
  'title': "Here's Why 

# 주식 데이터 수집
- stock_list: 종목코드, 종목명, 산업군
- stock_daily_info: 종목코드, 날짜, 종가, 거래수량, 거래대금, 등락률, 시가총액, 배당률, PER, 뉴스요약

## pymysql 설정

In [153]:
load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DB_CHARSET = os.getenv("DB_CHARSET")
def mysql_connection(host, user, password, db, charset):
    return pymysql.connect(host=host, user=user, password=password, db=db, charset=charset)

CONNECTION = mysql_connection(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME, 'utf8')
CONNECTION

### Gemini model 로드

In [133]:
load_dotenv()
GEM_API_KEY = os.getenv("GEM_API_KEY")

genai.configure(api_key=GEM_API_KEY)
genai_config = genai.GenerationConfig(max_output_tokens = 300, temperature = 0.5)
MODEL = genai.GenerativeModel(model_name='gemini-pro', generation_config = genai_config)

## 나스닥 종목 리스트 수집
- 후에 산업군 분석할 때 사용하기 위함

In [19]:
nasdaq_list = fdr.StockListing("NASDAQ")
nasdaq_list

100%|██████████████████████████████████████| 3738/3738 [00:04<00:00, 920.95it/s]


,Symbol,Name,IndustryCode,Industry
0,AAPL,Apple Inc,57106020,전화 및 소형 장치
1,MSFT,Microsoft Corp,57201020,소프트웨어
2,NVDA,NVIDIA Corp,57101010,반도체
3,AMZN,Amazon.com Inc,53402010,백화점
4,META,Meta Platforms Inc,57201030,온라인 서비스
...,...,...,...,...
3733,NEWTG,NewtekOne 8 50 Fixed Rate Senior Notes due 2029,55101050,기업 금융 서비스
3734,NYMTI,New York Mortgage Trust Inc 9 125 Senior Notes...,60102040,특수 REITs
3735,RFAIR,RF Acquisition II Right 01st May 2026,55601010,투자 지주 회사
3736,MBAVU,M3 Brigade Acquisition V Units,55601010,투자 지주 회사


In [20]:
nasdaq_list = list(nasdaq_list['Symbol'])
nasdaq_list

['AAPL',
 'MSFT',
 'NVDA',
 'AMZN',
 'META',
 'GOOGL',
 'GOOG',
 'TSLA',
 'AVGO',
 'COST',
 'ASML',
 'NFLX',
 'AMD',
 'ADBE',
 'AZN',
 'PEP',
 'QCOM',
 'LIN',
 'TMUS',
 'CSCO',
 'TXN',
 'PDD',
 'AMAT',
 'AMGN',
 'INTU',
 'ARM',
 'ISRG',
 'CMCSA',
 'INTC',
 'HON',
 'BKNG',
 'SNY',
 'VRTX',
 'MU',
 'LRCX',
 'ADI',
 'REGN',
 'KLAC',
 'PANW',
 'ADP',
 'ABNB',
 'GILD',
 'SNPS',
 'MDLZ',
 'SBUX',
 'MELI',
 'CDNS',
 'CTAS',
 'EQIX',
 'NXPI',
 'CME',
 'MAR',
 'CSX',
 'COIN',
 'CRWD',
 'PYPL',
 'ORLY',
 'ROP',
 'WDAY',
 'CEG',
 'NTES',
 'MRVL',
 'PCAR',
 'ADSK',
 'MNST',
 'IBKR',
 'CPRT',
 'MCHP',
 'AEP',
 'TTD',
 'ROST',
 'MRNA',
 'SMCI',
 'TEAM',
 'CHTR',
 'FTNT',
 'KDP',
 'DXCM',
 'PAYX',
 'DASH',
 'ODFL',
 'JD',
 'MPWR',
 'DDOG',
 'KHC',
 'VRSK',
 'IDXX',
 'FAST',
 'EA',
 'CTSH',
 'GEHC',
 'FANG',
 'NDAQ',
 'ACGL',
 'EXC',
 'BKR',
 'LULU',
 'CCEP',
 'ON',
 'BIIB',
 'BIDU',
 'MSTR',
 'FER',
 'CDW',
 'GFS',
 'XEL',
 'CSGP',
 'TCOM',
 'ALNY',
 'TSCO',
 'ZS',
 'ICLR',
 'FITB',
 'ANSS',
 'APP',


In [134]:
class StockInfo:
    def __init__(self, ticker):
        self.ticker = ticker
        self.data = yf.Ticker(ticker).info
        self.history = yf.Ticker(ticker).history(period="1mo")
        self.today = datetime.now().strftime("%Y-%m-%d")

    def get_name(self):
        try:
            return self.data["shortName"]
        except:
            print(f"{self.ticker} get_name() Error")
            return None

    def get_industry(self):
        try:
            industry = f'{self.data["industry"]}/{self.data["sector"]}'
            industry = industry.replace("—", " - ")
            industry = industry.replace("&", "n")
            return industry
        except:
            print(f"{self.ticker} get_industry() Error")
            return None
    
    def get_price(self):
        try:
            return self.data["currentPrice"]
        except:
            print(f"{self.ticker} get_price() Error")
            return None

    def get_dividend_rate(self):
        try:
            return round(self.data["dividendRate"], 2)
        except:
            print(f"{self.ticker} get_dividend_rate() Error")
            return None

    def get_dividend_yield(self):
        try:
            return round(self.data["dividendYield"], 2)
        except:
            print(f"{self.ticker} get_dividend_yield() Error")
            return None

    def get_volume(self):
        try:
            return self.data["volume"]
        except:
            print(f"{self.ticker} get_volume() Error")
            return None

    def get_avg_volume(self):
        try:
            return self.data["averageVolume"]
        except:
            print(f"{self.ticker} get_avg_volume() Error")
            return None


    def get_market_cap(self):
        try:
            return round(self.data["marketCap"], 2)
        except:
            print(f"{self.ticker} get_market_cap() Error")
            return None

    def get_per(self):
        try:
            return round(self.data["trailingPE"], 2)
        except:
            print(f"{self.ticker} get_per() Error")
            return None

    def get_fluc(self):
        try:
            prev = self.history.iloc[-2]["Close"]
            cur = self.history.iloc[-1]["Close"]
            return round((cur - prev) / prev * 100, 2)
        except (IndexError, KeyError):
            print(f"{self.ticker} get_fluc() Error")
            return None

    def insert_data(self, connection, query, *args):
        cursor = connection.cursor(pymysql.cursors.DictCursor)
        cursor.execute(query, args)
        connection.commit()
        cursor.close()

In [135]:
def stock_list(nasdaq_list):
    for ticker in nasdaq_list:
        stock = StockInfo(ticker)

        name = stock.get_name()
        if name is None:
            print(f"{ticker} name이 존재하지 않음")
            continue

        industry = stock.get_industry()
        if industry is None:
            print(f"{ticker} industry가 존재하지 않음")
            continue

        stock.insert_data(CONNECTION, query1, ticker, name, industry)

    print("데이터 삽입 완료!")

In [136]:
class NewsSummary:
    def __init__(self, ticker):
        self.ticker = ticker
            
    def get_urls(self):
        urls = []
        headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
        }
        for i in range(1, 3):
            # url = f"https://www.investing.com/equities/{companies[self.ticker]}-news/{i}"
            url = f"https://www.investors.com/search-results/?query={self.ticker}"
            response = requests.get(url, headers={'User-agent':'Mozila/6.0'})
            soup = BeautifulSoup(response.text, 'html.parser')
            print(soup.prettify())
            articles = soup.select('div[class="queryly_item_row"]')
            print(articles)
            for article in articles:
                pro = article.select_one('svg[height="17.5"]')
                if not pro:
                    title_element = article.select_one('a[data-test="article-title-link"]')
                    href = title_element.get('href')
                    link = "http://www.investing.com"+href
                    urls.append(link)

        print(f"{self.ticker}의 뉴스기사 {len(urls)}개를 확인합니다.")
        return urls

    def get_written_time(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        wtime = soup.select('div[class="flex flex-col gap-2 text-warren-gray-700 md:flex-row md:items-center md:gap-0"] div span')

        if not wtime:
            return None
        wtime = wtime[0].text[10:]
        
        if wtime[-2:] == 'AM':
            year = wtime[6:10]
            month = wtime[:2]
            day = wtime[3:5]
            hour = wtime[12:14]
            minute = wtime[15:17]
            wtime = datetime.strptime(f"{year}-{month}-{day} {hour}:{minute}", "%Y-%m-%d %H:%M")
        else:
            year = wtime[6:10]
            month = wtime[:2]
            day = wtime[3:5]
            if int(wtime[12:14]) == 12:
                hour = wtime[12:14]
            else:
                hour = str(int(wtime[12:14])+12)
            minute = wtime[15:17]
            wtime = datetime.strptime(f"{year}-{month}-{day} {hour}:{minute}", "%Y-%m-%d %H:%M")
        return wtime

    def get_article(self, wtime, url):
        print(f"{wtime}에 작성된 기사를 긁어옵니다.")
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # article
        # 'div[id="article"]' 아래의 모든 p 태그를 찾음
        all_p_tags = soup.select('div[id="article"] p')
        
        # 'blockquote' 태그 내의 p 태그를 찾음
        blockquote_p_tags = soup.select('div[id="article"] blockquote p')
        
        # 'blockquote' 태그 내의 p 태그를 제외한 p 태그를 필터링
        filtered_p_tags = [p for p in all_p_tags if p not in blockquote_p_tags]
    
        article = "\n".join([text.text for text in filtered_p_tags if text.name != 'span'])

        return article

    def summarize_article(self, ticker, article):
        prompt = f"""
        페르소나: 주식 전문가
        목적: 뉴스 요약

        아래에 주어지는 뉴스기사를 한국어로 요약해줘.
        요약 전, 몇가지 지켜줘야할 사항들은 다음과 같아.
        1. {ticker}에 대한 사건사고나 이벤트를 명확하고 구체적으로 언급해줘.
        2. "InvestingPro"와 같은 광고성 내용이 포함되어 있다면 생략해줘
        3. 별도의 문장 없이 곧바로 요약문만 생성해줘
        4. 각 문장은 반드시 개행되어야해.(= "\n"로 끝나야해)
        5. 답변은 300 토큰을 초과해서는 안돼.

        뉴스기사:
        {article}
        """
        try:
            summary = MODEL.generate_content(prompt).text
            return summary

        except Exception as e:
            return "gemini 오류 발생"

In [137]:
class News:
    def __init__(self, ticker):
        self.ticker = ticker

    def get_news(self):
        news_list = []
        try:
            news = yf.Ticker(self.ticker).news
            for item in news:
                title = item['title']
                link = item['link']
                thumbnail_resolutions = item.get('thumbnail', {}).get('resolutions', [])
                thumbnail = thumbnail_resolutions[1]['url'] if len(thumbnail_resolutions) > 1 else None
                if title and link and thumbnail:
                    news_list.append([title, link, thumbnail])
            return news_list if news_list else None
        except:
            print(f"{self.ticker} get_news() Error")

In [138]:
def stock_daily_info(nasdaq_list):
    for ticker in nasdaq_list:
        stock = StockInfo(ticker)

        price = stock.get_price()
        if price is None:
            price = 0

        dividend_rate = stock.get_dividend_rate()
        if dividend_rate is None:
            divident_rate = 0
        
        dividend_yield = stock.get_dividend_yield()
        if dividend_yield is None:
            dividend_yield = 0
        
        volume = stock.get_volume()
        if volume is None:
            volume = 0
        
        avg_volume = stock.get_avg_volume()
        if avg_volume is None:
            avg_volume = 0
        
        market_cap = stock.get_market_cap()
        if market_cap is None:
            market_cap = 0
        
        per = stock.get_per()
        if per is None:
            per = 0
        
        fluc = stock.get_fluc()
        if fluc is None:
            fluc = 0

        news_extractor = News(ticker)
        news_list = news_extractor.get_news()
        if news_list is None:
            title = f"오늘은 {ticker} 관련 뉴스가 없습니다!"
            stock.insert_data(CONNECTION, query2, stock.today, ticker, price, dividend_rate, dividend_yield, volume, avg_volume, market_cap, per, fluc, title, "None", "None")
            print(f"{ticker} DB에 저장합니다.")
        else:
            for news in news_list:
                title = news[0]
                link = news[1]
                thumbnail = news[2]
                
                stock.insert_data(CONNECTION, query2, stock.today, ticker, price, dividend_rate, dividend_yield, volume, avg_volume, market_cap, per, fluc, title, link, thumbnail)
                print(f"{ticker} DB에 저장합니다.")
        print(f"{ticker} DB 저장 완료!")
            
    #    # 뉴스요약
    #     summarizer = NewsSummary(ticker)
    #     urls = summarizer.get_urls()
    #     summaries = []
    #     count = 0
    #     for idx, url in enumerate(urls):
    #         wtime = summarizer.get_written_time(url)
    #         if wtime != None:
    #             year = stock.today[:4]
    #             month = stock.today[5:7]
    #             day = stock.today[8:10]

    #             # previous_close_time = datetime.strptime(f"{year}-{month}-{day} 16:00", "%Y-%m-%d %H:%M") - timedelta(days=1) # 전 날 폐장시간
    #             crawling_time = datetime.strptime(f"{year}-{month}-{day} 5:00", "%Y-%m-%d %H:%M") # 당일 개장 두시간 전
                
    #             # if previous_close_time <= wtime <= crawling_time:
    #             if crawling_time - timedelta(days=1) <= wtime <= crawling_time:
    #                 count += 1
    #                 article = summarizer.get_article(wtime, url)
    #                 # s3 적재
    #                 summary = summarizer.summarize_article(ticker, article)
    #                 summaries.append(summary)

    #     if count == 0:
    #         summary = "None"
    #         stock.insert_data(CONNECTION, query2, stock.today, ticker, price, dividend_rate, dividend_yield, volume, avg_volume, market_cap, per, fluc, summary)
    #         print(f"{ticker} DB에 저장합니다.")
    #     else:
    #         for summary in summaries:
    #             stock.insert_data(CONNECTION, query2, stock.today, ticker, price, dividend_rate, dividend_yield, volume, avg_volume, market_cap, per, fluc, summary)
    #             print(f"{ticker} DB에 저장합니다.")
    # print("데이터 삽입 완료!")

## 나스닥100 종목 리스트
- 매일 바뀌는 리스트이긴 하지만, 현재는 개발의 편의를 위해 고정된 리스트 사용
- names는 investing.com 크롤링할 때 url에 들어가는 매개변수.
    - 무작위 크롤링 방지 목적인지 url매개변수가 ticker로 안되어있고 이상한 이름으로 되어있어 수작업으로 모을 수 밖에 없음ㅠ
    - yfinance API쓸까 고민했지만, 뉴스기사 크롤링에 어려움이 있어 그냥 investing.com 사용

In [139]:
# Investing.com 뉴스 크롤링용 Ticker, name
companies = dict()
nasdaq100_tickers = ['MSFT','AAPL','NVDA','GOOG','GOOGL','AMZN','META','AVGO','TSLA','ASML','COST','PEP','NFLX','AZN','AMD','LIN','ADBE','TMUS',
 'CSCO','QCOM','INTU','PDD','AMAT','TXN','CMCSA','AMGN','ISRG','INTC','HON','MU','BKNG','LRCX','VRTX','ADP','REGN','ABNB','ADI','MDLZ',
 'PANW','KLAC','SBUX','GILD','SNPS','CDNS','MELI','CRWD','PYPL','MAR','CTAS','CSX','WDAY','NXPI','ORLY','CEG','PCAR','MNST','MRVL','ROP',
 'CPRT','DASH','DXCM','FTNT','MCHP','AEP','KDP','ADSK','TEAM','LULU','KHC','PAYX','ROST','MRNA','DDOG','TTD','ODFL','FAST','IDXX','EXC',
 'CHTR','CSGP','GEHC','FANG','EA','VRSK','CCEP','CTSH','BKR','BIIB','XEL','ON','CDW','ANSS','MDB','DLTR','ZS','GFS','TTWO','ILMN','WBD',
 'WBA','SIRI']
names = [
         "microsoft-corp", "apple-computer-inc", "nvidia-corp", "google-inc-c", "google-inc", "amazon-com-inc", "facebook-inc",
         "avago-technologies", "tesla-motors", "asml-holdings", "costco-whsl-corp-new", "pepsico", "netflix,-inc.", "astrazeneca-plc-ads",
         "adv-micro-device", "linde-plc", "adobe-sys-inc", "metropcs-communications", "cisco-sys-inc", "qualcomm-inc", "intuit", "pinduoduo",
         "applied-matls-inc", "texas-instru", "comcast-corp-new", "amgen-inc", "intuitive-surgical-inc", "intel-corp", "honeywell-intl",
         "micron-tech", "priceline.com-inc", "lam-research-corp", "vertex-pharm", "auto-data-process", "regeneron-phar.", "airbnb-inc",
         "analog-devices", "mondelez-international-inc", "palo-alto-netwrk", "kla-tencor-corp", "starbucks-corp", "gilead-sciences-inc",
         "synopsys-inc", "cadence-design-system-inc", "mercadolibre", "crowdstrike-holdings-inc", "paypal-holdings-inc", "marriott-intl",
         "cintas-corp", "csx-corp", "workday-inc", "nxp-semiconductors", "oreilly-automotive", "constellation-energy", "paccar-inc",
         "monster-beverage", "marvell-technology-group-ltd", "roper-industries",  "copart-inc", "doordash-inc",
         "dexcom", "fortinet", "microchip-technology-inc", "american-electric", "dr-pepper-snapple", "autodesk-inc", "atlassian-corp-plc",
         "lululemon-athletica", "kraft-foods-inc", "paychex-inc", "ross-stores-inc", "moderna", "datadog-inc", "trade-desk-inc",
         "old-dominion-freight-line-inc", "fastenal-co", "idexx-laboratorie", "exelon-corp",
         "charter-communications", "costar-group", "ge-healthcare-holding-llc", "diamondback-energy-inc",
         "electronic-arts-inc", "verisk-analytics-inc", "coca-cola-ent", "cognizant-technology-solutio", "baker-hughes", "biogen-idec-inc",
         "xcel-energy", "on-semiconductor", "cdw-corp", "ansys", "mongodb", "dollar-tree-inc", "zscaler-inc", "globalfoundries",
         "take-two-interactive", "illumina,-inc.", "discovery-holding-co", "walgreen-co", "sirius-satellite-radio-inc"
        ]
for i in range(101):
    companies[nasdaq100_tickers[i]] = names[i]

companies

{'MSFT': 'microsoft-corp',
 'AAPL': 'apple-computer-inc',
 'NVDA': 'nvidia-corp',
 'GOOG': 'google-inc-c',
 'GOOGL': 'google-inc',
 'AMZN': 'amazon-com-inc',
 'META': 'facebook-inc',
 'AVGO': 'avago-technologies',
 'TSLA': 'tesla-motors',
 'ASML': 'asml-holdings',
 'COST': 'costco-whsl-corp-new',
 'PEP': 'pepsico',
 'NFLX': 'netflix,-inc.',
 'AZN': 'astrazeneca-plc-ads',
 'AMD': 'adv-micro-device',
 'LIN': 'linde-plc',
 'ADBE': 'adobe-sys-inc',
 'TMUS': 'metropcs-communications',
 'CSCO': 'cisco-sys-inc',
 'QCOM': 'qualcomm-inc',
 'INTU': 'intuit',
 'PDD': 'pinduoduo',
 'AMAT': 'applied-matls-inc',
 'TXN': 'texas-instru',
 'CMCSA': 'comcast-corp-new',
 'AMGN': 'amgen-inc',
 'ISRG': 'intuitive-surgical-inc',
 'INTC': 'intel-corp',
 'HON': 'honeywell-intl',
 'MU': 'micron-tech',
 'BKNG': 'priceline.com-inc',
 'LRCX': 'lam-research-corp',
 'VRTX': 'vertex-pharm',
 'ADP': 'auto-data-process',
 'REGN': 'regeneron-phar.',
 'ABNB': 'airbnb-inc',
 'ADI': 'analog-devices',
 'MDLZ': 'mondelez-in

### stock_list 테이블 삽입

In [17]:
query1 = """
INSERT INTO stock_list (ticker, name, industry) VALUES (%s, %s, %s);
"""
stock_list(nasdaq100_tickers)

IntegrityError: (1062, "Duplicate entry 'MSFT' for key 'stock_list.PRIMARY'")

### stock_daily_info 테이블 삽입

In [140]:
query2 = """
INSERT INTO stock_daily_info (date, ticker, price, dividend_rate, dividend_yield, volume, avg_volume, market_cap, per, pct_change, news_title, news_link, news_thumbnail) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
stock_daily_info(nasdaq100_tickers)

MSFT DB에 저장합니다.
MSFT DB에 저장합니다.
MSFT DB에 저장합니다.
MSFT DB에 저장합니다.
MSFT DB에 저장합니다.
MSFT DB에 저장합니다.
MSFT DB 저장 완료!
AAPL DB에 저장합니다.
AAPL DB에 저장합니다.
AAPL DB에 저장합니다.
AAPL DB에 저장합니다.
AAPL DB에 저장합니다.
AAPL DB에 저장합니다.
AAPL DB에 저장합니다.
AAPL DB에 저장합니다.
AAPL DB 저장 완료!
NVDA DB에 저장합니다.
NVDA DB에 저장합니다.
NVDA DB에 저장합니다.
NVDA DB에 저장합니다.
NVDA DB에 저장합니다.
NVDA DB에 저장합니다.
NVDA DB에 저장합니다.
NVDA DB에 저장합니다.
NVDA DB 저장 완료!
GOOG DB에 저장합니다.
GOOG DB에 저장합니다.
GOOG DB에 저장합니다.
GOOG DB에 저장합니다.
GOOG DB에 저장합니다.
GOOG DB에 저장합니다.
GOOG DB에 저장합니다.
GOOG DB 저장 완료!
GOOGL DB에 저장합니다.
GOOGL DB에 저장합니다.
GOOGL DB에 저장합니다.
GOOGL DB에 저장합니다.
GOOGL DB에 저장합니다.
GOOGL DB에 저장합니다.
GOOGL DB에 저장합니다.
GOOGL DB 저장 완료!
AMZN get_dividend_rate() Error
AMZN get_dividend_yield() Error
AMZN DB에 저장합니다.
AMZN DB에 저장합니다.
AMZN DB에 저장합니다.
AMZN DB에 저장합니다.
AMZN DB에 저장합니다.
AMZN DB에 저장합니다.
AMZN DB에 저장합니다.
AMZN DB 저장 완료!
META DB에 저장합니다.
META DB에 저장합니다.
META DB에 저장합니다.
META DB에 저장합니다.
META DB에 저장합니다.
META DB에 저장합니다.
META DB에 저장합니다.
META DB에 저장합니다.
META DB 저장 완료!
AVGO DB에

## 볼린저밴드용 주가데이터 수집

In [142]:
class BollingerBand:
    def __init__(self, ticker):
        self.ticker = ticker
        self.today = datetime.now().strftime("%Y-%m-%d")

    def read_stock_data(self):
        data = yf.Ticker(self.ticker)
        df = data.history(period="2y")
        df = df.reset_index()
        df = df[["Date", "Open", "High", "Low", "Close"]]
        df = df.assign(Ticker=self.ticker, axis=0)
        df['ma20'] = df['Close'].rolling(window=20).mean()
        df['std'] = df['Close'].rolling(window=20).std()
        df['upper'] = df['ma20'] + (df['std'] * 2)
        df['lower'] = df['ma20'] - (df['std'] * 2)
        df = df.fillna(value=0)
        df_ohlc = df[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'ma20', 'std', 'upper', 'lower']]
        df_ohlc['Date'] = pd.to_datetime(df_ohlc['Date'])
        df_ohlc['Date'] = df_ohlc['Date'].astype('int64') // 10**6
        return df_ohlc

    def insert_data(self, connection, query, df):
        with connection.cursor() as cursor:
            # 데이터베이스가 비어있는지 확인
            cursor.execute("SELECT COUNT(*) FROM stock_chart_info WHERE ticker = %s", self.ticker)
            result = cursor.fetchone()

            # 데이터베이스가 비어있는 경우
            if result[0] == 0:
                # 판다스 데이터프레임의 모든 행을 insert
                for idx, row in df.iterrows():
                    cursor.execute(query, (row['Ticker'], row['Date'], row['Open'], 
                                     row['High'], row['Low'], row['Close'], 
                                     row['ma20'], row['std'], row['upper'], row['lower']))
            # 데이터베이스에 해당 종목이 이미 있을 경우
            else:
                # 판다스 데이터프레임의 마지막 행만 insert
                last_row = df.iloc[-1]
                cursor.execute(query, (last_row['Ticker'], last_row['Date'], last_row['Open'], 
                                 last_row['High'], last_row['Low'], last_row['Close'],
                                last_row['ma20'], last_row['std'], last_row['upper'], last_row['lower']))
            # 변경사항 커밋
            connection.commit()

In [143]:
def stock_chart_info(nasdaq_list):
    for ticker in nasdaq_list:
        stock = BollingerBand(ticker)
        df = stock.read_stock_data()
        stock.insert_data(CONNECTION, query3, df)
        print(f"{ticker} 데이터 삽입 완료")
    print("데이터 삽입 완료!")

### stock_daily_info 테이블 삽입

In [144]:
query3 = """
INSERT INTO stock_chart_info (ticker, date, open_price, high_price, low_price, close_price, ma20, std, upper, lower) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

stock_chart_info(nasdaq100_tickers)

MSFT 데이터 삽입 완료
AAPL 데이터 삽입 완료
NVDA 데이터 삽입 완료
GOOG 데이터 삽입 완료
GOOGL 데이터 삽입 완료
AMZN 데이터 삽입 완료
META 데이터 삽입 완료
AVGO 데이터 삽입 완료
TSLA 데이터 삽입 완료
ASML 데이터 삽입 완료
COST 데이터 삽입 완료
PEP 데이터 삽입 완료
NFLX 데이터 삽입 완료
AZN 데이터 삽입 완료
AMD 데이터 삽입 완료
LIN 데이터 삽입 완료
ADBE 데이터 삽입 완료
TMUS 데이터 삽입 완료
CSCO 데이터 삽입 완료
QCOM 데이터 삽입 완료
INTU 데이터 삽입 완료
PDD 데이터 삽입 완료
AMAT 데이터 삽입 완료
TXN 데이터 삽입 완료
CMCSA 데이터 삽입 완료
AMGN 데이터 삽입 완료
ISRG 데이터 삽입 완료
INTC 데이터 삽입 완료
HON 데이터 삽입 완료
MU 데이터 삽입 완료
BKNG 데이터 삽입 완료
LRCX 데이터 삽입 완료
VRTX 데이터 삽입 완료
ADP 데이터 삽입 완료
REGN 데이터 삽입 완료
ABNB 데이터 삽입 완료
ADI 데이터 삽입 완료
MDLZ 데이터 삽입 완료
PANW 데이터 삽입 완료
KLAC 데이터 삽입 완료
SBUX 데이터 삽입 완료
GILD 데이터 삽입 완료
SNPS 데이터 삽입 완료
CDNS 데이터 삽입 완료
MELI 데이터 삽입 완료
CRWD 데이터 삽입 완료
PYPL 데이터 삽입 완료
MAR 데이터 삽입 완료
CTAS 데이터 삽입 완료
CSX 데이터 삽입 완료
WDAY 데이터 삽입 완료
NXPI 데이터 삽입 완료
ORLY 데이터 삽입 완료
CEG 데이터 삽입 완료
PCAR 데이터 삽입 완료
MNST 데이터 삽입 완료
MRVL 데이터 삽입 완료
ROP 데이터 삽입 완료
CPRT 데이터 삽입 완료
DASH 데이터 삽입 완료
DXCM 데이터 삽입 완료
FTNT 데이터 삽입 완료
MCHP 데이터 삽입 완료
AEP 데이터 삽입 완료
KDP 데이터 삽입 완료
ADSK 데이터 삽입 완료
TEAM 데이터 삽입 완료
LULU 데이터 삽

## 산업군 분석용 데이터 수집

In [154]:
class IndustryAnalysis:
    def __init__(self, df):
        self.df = df
        self.today = datetime.now().strftime("%Y-%m-%d")
        
    def get_unique_industries(self):
        industry_list = list(self.df['industry'].unique())
        return industry_list

    def remove_outliers_and_insert_data(self, industry_list, connection, query):
        for industry in industry_list:
            avg_market_cap = round(self.df[self.df['industry']== industry]['market_cap'].mean())

            per_values = np.array(self.df[self.df['industry'] == industry]['per'])
            perQ1 = np.percentile(per_values, 25)
            perQ3 = np.percentile(per_values, 75)
            perIQR = perQ3 - perQ1
            per_lower_bound = perQ1 - 1.5*perIQR
            per_upper_bound = perQ3 + 1.5*perIQR
            avg_per = round(per_values[(per_values >= per_lower_bound) & (per_values <= per_upper_bound)].mean(), 2)

            avg_pct_change = round(self.df[self.df['industry'] == industry]['pct_change'].mean(), 2)

            self.insert_data(connection, query, industry, self.today, avg_market_cap, avg_per, avg_pct_change)

    def insert_data(self, connection, query, *args):
        cursor = connection.cursor(pymysql.cursors.DictCursor)
        cursor.execute(query, args)
        connection.commit()
        cursor.close()
        

In [155]:
def industry_info(nasdaq_list):
    nasdaq_dict = {}
    for idx, ticker in enumerate(nasdaq_list, start=1):
        stock = StockInfo(ticker)

        name = stock.get_name()
        if name is None:
            continue

        industry = stock.get_industry()
        if industry is None:
            continue

        market_cap = stock.get_market_cap()
        if market_cap is None:
            continue

        per = stock.get_per()
        if per is None:
            continue

        fluc = stock.get_fluc()
        if fluc is None:
            continue

        print(f"{idx} / {len(nasdaq_list)}")
        nasdaq_dict[ticker] = [name, industry, market_cap, per, fluc]

    df = pd.DataFrame.from_dict(data=nasdaq_dict, orient='index', columns=['name', 'industry', 'market_cap', 'per', 'pct_change'])
    df = df.reset_index().rename(columns={"index":"ticker"})
    df['per'] = df['per'].astype('float64')
    df['pct_change'] = df['pct_change'].astype('float64')
    # inf를 포함하는 행은 삭제
    df = df.drop(df[df['per']==np.inf].index).reset_index(drop=True)

    industry_analyzer = IndustryAnalysis(df)
    industry_list = industry_analyzer.get_unique_industries()
    industry_analyzer.remove_outliers_and_insert_data(industry_list, CONNECTION, query4)

In [156]:
query4 = """
INSERT INTO industry_info (industry, date, avg_market_cap, avg_per, avg_pct_change) VALUES (%s, %s, %s, %s, %s)
"""
industry_info(nasdaq_list)

1 / 3738
2 / 3738
3 / 3738
4 / 3738
5 / 3738
6 / 3738
7 / 3738
8 / 3738
9 / 3738
10 / 3738
11 / 3738
12 / 3738
13 / 3738
14 / 3738
15 / 3738
16 / 3738
17 / 3738
18 / 3738
19 / 3738
20 / 3738
21 / 3738
22 / 3738
23 / 3738
24 / 3738
25 / 3738
26 / 3738
27 / 3738
28 / 3738
29 / 3738
30 / 3738
31 / 3738
32 / 3738
33 / 3738
MU get_per() Error
35 / 3738
36 / 3738
37 / 3738
38 / 3738
39 / 3738
40 / 3738
41 / 3738
42 / 3738
43 / 3738
44 / 3738
45 / 3738
46 / 3738
47 / 3738
48 / 3738
49 / 3738
50 / 3738
51 / 3738
52 / 3738
53 / 3738
54 / 3738
55 / 3738
56 / 3738
57 / 3738
58 / 3738
59 / 3738
60 / 3738
61 / 3738
MRVL get_per() Error
63 / 3738
64 / 3738
65 / 3738
66 / 3738
67 / 3738
68 / 3738
69 / 3738
70 / 3738
71 / 3738
MRNA get_per() Error
73 / 3738
TEAM get_per() Error
75 / 3738
76 / 3738
77 / 3738
78 / 3738
79 / 3738
DASH get_per() Error
81 / 3738
82 / 3738
83 / 3738
84 / 3738
85 / 3738
86 / 3738
87 / 3738
88 / 3738
89 / 3738
90 / 3738
91 / 3738
92 / 3738
93 / 3738
94 / 3738
95 / 3738
96 / 3

$FUSN: possibly delisted; No price data found  (period=1mo)


$FUSN: possibly delisted; No price data found  (period=1mo)
FUSN get_per() Error
UPBD get_per() Error
745 / 3738
DNUT get_per() Error
747 / 3738
748 / 3738
749 / 3738
VCYT get_per() Error
751 / 3738
EXTR get_per() Error
753 / 3738
754 / 3738
755 / 3738
PSNY get_per() Error
757 / 3738
758 / 3738
KROS get_per() Error
EVO get_per() Error
OPEN get_per() Error
762 / 3738
SONO get_per() Error
764 / 3738
765 / 3738
766 / 3738
767 / 3738
768 / 3738
NAMS get_per() Error
PWP get_per() Error
771 / 3738
VIAV get_per() Error
773 / 3738
774 / 3738
775 / 3738
SUPN get_per() Error
777 / 3738
778 / 3738
SOUN get_per() Error
SLNO get_per() Error
SDGR get_per() Error
782 / 3738
783 / 3738
784 / 3738
785 / 3738
786 / 3738
787 / 3738
788 / 3738
789 / 3738
BTDR get_per() Error
791 / 3738
792 / 3738
793 / 3738
TLRY get_per() Error
AVDL get_per() Error
KURA get_per() Error
LILAK get_per() Error
798 / 3738
SYRE get_per() Error
800 / 3738
COHU get_per() Error
802 / 3738
803 / 3738
804 / 3738
805 / 3738
STER get

$VTRU: possibly delisted; No price data found  (period=1mo)


$VTRU: possibly delisted; No price data found  (period=1mo)
VTRU get_fluc() Error
1603 / 3738
1604 / 3738
1605 / 3738
AFRI get_per() Error
VRCA get_per() Error
1608 / 3738
1609 / 3738
1610 / 3738
IVCAU get_market_cap() Error
1612 / 3738
ONYXU get_market_cap() Error
BLEUU get_market_cap() Error
ALIM get_per() Error
CCIXU get_industry() Error
1617 / 3738
GMGI get_per() Error
QRTEA get_per() Error
AQST get_per() Error
SERV get_per() Error
1622 / 3738
SSP get_per() Error
TLS get_per() Error
RNAC get_per() Error
MGTX get_per() Error
1627 / 3738
1628 / 3738
1629 / 3738
OMER get_per() Error
ACCD get_per() Error
LFMD get_per() Error
MTRX get_per() Error
1634 / 3738
AKBA get_per() Error
GPATU get_market_cap() Error
1637 / 3738
TNYA get_per() Error
1639 / 3738
1640 / 3738
BLDE get_per() Error
MYPS get_per() Error
1643 / 3738
OBIO get_per() Error
ANGO get_per() Error
WTFCP get_market_cap() Error
ZNTL get_per() Error
1648 / 3738
EMLDU get_market_cap() Error
1650 / 3738
1651 / 3738
1652 / 3738
1653

...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SCTL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SCTL&crumb=GZh2xQYBos1
SCTL: No data found, symbol may be delisted


SCTL get_name() Error
2147 / 3738
RFAIU get_industry() Error
SHIM get_per() Error
MPAA get_per() Error


$TWLV: possibly delisted; No price data found  (period=1mo)


$TWLV: possibly delisted; No price data found  (period=1mo)
TWLV get_fluc() Error
AVHIU get_market_cap() Error
2153 / 3738
THCH get_per() Error
HOWL get_per() Error
BSET get_per() Error
2157 / 3738
OFS get_per() Error
AMLI get_per() Error
IVP get_per() Error
ATYR get_per() Error
ALLT get_per() Error
TKNO get_per() Error
KRMD get_per() Error
2165 / 3738
TELO get_per() Error
SEER get_per() Error
2168 / 3738
2169 / 3738
INSG get_per() Error
2171 / 3738
HTLFP get_market_cap() Error
AOUT get_per() Error
NYMTL get_market_cap() Error
2175 / 3738
AKYA get_per() Error
2177 / 3738
GNSS get_per() Error
ADAG get_per() Error
2180 / 3738
BCOV get_per() Error
2182 / 3738
MSBIP get_market_cap() Error
2184 / 3738
2185 / 3738
IMAB get_per() Error
OPRT get_per() Error
2188 / 3738
KPTI get_per() Error
RFACU get_market_cap() Error
2191 / 3738
CITE get_per() Error
2193 / 3738
CFFS get_per() Error
ROCLU get_market_cap() Error
HURC get_per() Error
XTKG get_per() Error
RTC get_per() Error
2199 / 3738
WINVU get

ReadTimeout: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=30)